#  Прогнозирование спроса на аренду велосипедов: Neural Networks vs Classical Models

##  Постановка задачи
**Дело:** Регрессионное предсказание почасового спроса (`cnt`) на велосипеды в системе Capital Bikeshare (Вашингтон, 2011-2012).  
**Цель:** Демонстрация условий, при которых **нейронные сети превосходят классические алгоритмы** (LinearRegression, RandomForest).  

**Зачем это нужно:** Bike Sharing Dataset (17,379 наблюдений × 14 фич) содержит **сложные нелинейные зависимости** (час × погода × сезон), идеальные для глубоких сетей.

##  Ожидаемый результат
**Гипотеза:** На 17k сэмплах с циклическим кодированием NN даст **R²=95%+** против RF ~94%.

---

## 1. Загрузка и обзор датасета


In [20]:
# Загрузка Bike Sharing Dataset (UCI ID=275)
import ucimlrepo
import pandas as pd
import numpy as np

bike = ucimlrepo.fetch_ucirepo(id=275)
df = pd.concat([bike.data.features, bike.data.targets], axis=1)

print("=== ОСНОВНЫЕ ХАРАКТЕРИСТИКИ ===")
print(f"Размер: {df.shape}")
print("\nTarget (cnt):")
print(df['cnt'].describe())
print("\nИнформация о колонках:")
print(df.info())
print("\nПервые 5 строк:")
print(df.head())


=== ОСНОВНЫЕ ХАРАКТЕРИСТИКИ ===
Размер: (17379, 14)

Target (cnt):
count    17379.000000
mean       189.463088
std        181.387599
min          1.000000
25%         40.000000
50%        142.000000
75%        281.000000
max        977.000000
Name: cnt, dtype: float64

Информация о колонках:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dteday      17379 non-null  object 
 1   season      17379 non-null  int64  
 2   yr          17379 non-null  int64  
 3   mnth        17379 non-null  int64  
 4   hr          17379 non-null  int64  
 5   holiday     17379 non-null  int64  
 6   weekday     17379 non-null  int64  
 7   workingday  17379 non-null  int64  
 8   weathersit  17379 non-null  int64  
 9   temp        17379 non-null  float64
 10  atemp       17379 non-null  float64
 11  hum         17379 non-null  float64
 12  windspeed   17379 n

<h1 id="dataset-overview" align="center" style="color:#000000;background:#FFFACD;padding:18px;border-radius:15px 50px; font-family:newtimeroman;">
 Обзор датасета Bike Sharing Dataset (UCI #275)
</h1>

<h2 id="data-quality" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
Качество и характеристики данных
</h2>

<table style="width:100%; border-collapse:collapse;">
<tr style="background:#90EE90;">
    <th>Характеристика</th><th>Значение</th><th>Вывод</th>
</tr>
<tr>
    <td><b>Размер датасета</b></td>
    <td><b class="highlight">17,379</b> почасовых записей</td>
    <td> <b>Идеальный объём для NN</b> — >10k сэмплов стабилизирует глубокие сети</td>
</tr>
<tr>
    <td><b>Количество фич</b></td>
    <td><b>14 колонок</b> (13 информативных)</td>
    <td> <b>Средняя размерность</b> — баланс сложности/обучаемости</td>
</tr>
<tr style="background:#FFF2CC;">
    <td><b>Missing values</b></td>
    <td><b>0</b> (все 17,379 non-null)</td>
    <td> <b>Чистые данные</b> — без предобработки пропусков</td>
</tr>
</table>

<h2 id="target-analysis" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
Анализ целевой переменной (cnt)
</h2>

<table style="width:100%; border-collapse:collapse;">
<tr style="background:#90EE90;">
    <th>Метрика</th><th>Значение</th><th>Вывод</th>
</tr>
<tr style="background:#FFF2CC;">
    <td><b>Right skew</b></td>
    <td>mean=189 | median=142 | max=977</td>
    <td> <b>Требует log1p()</b> — критично для всех моделей</td>
</tr>
<tr>
    <td><b>Диапазон спроса</b></td>
    <td>1-977 велосипедов/час</td>
    <td> <b>Реалистичные значения</b> — бизнес-задача</td>
</tr>
<tr>
    <td><b>Стандартное отклонение</b></td>
    <td>181 (высокая изменчивость)</td>
    <td> <b>Вызов для моделей</b> — сложная регрессия</td>
</tr>
</table>

<h2 id="features-overview" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
Обзор признаков
</h2>

<h3 style="color:#000000;background:#F0F8FF;padding:8px;border-radius:8px;font-family:monospace;"> ПОЛОЖИТЕЛЬНЫЕ характеристики</h3>
<ul>
<li><b>Нормализованные фичи:</b> temp/atemp/hum/windspeed ∈ [0,1] — идеально для NN</li>
<li><b>Цикличные переменные:</b> hr(0-23), mnth(1-12), weekday(0-6) → sin/cos encoding</li>
<li><b>Категориальные:</b> season(1-4), weathersit(1-4) — one-hot или ordinal</li>
<li><b>Бинарные:</b> holiday/workingday — готово к использованию</li>
</ul>

<h3 style="color:#000000;background:#F0F8FF;padding:8px;border-radius:8px;font-family:monospace;"> ТРЕБУЮТ обработки</h3>
<ul>
<li><code>dteday</code> — datetime → исключаем из модели</li>
<li><code>casual/registered</code> отсутствуют — cnt=(casual+registered)</li>
</ul>

<h1 id="readiness" align="center" style="color:#000000;background:#90EE90;padding:18px;border-radius:15px 50px; font-family:newtimeroman;">
 ГОТОВНОСТЬ К ОБУЧЕНИЮ: 95/100
</h1>

<h3 style="color:#000000;background:#F0F8FF;padding:12px;border-radius:8px;font-family:monospace;">
<h4> ПЛЮСЫ:</h4>
- Большой объём данных (17k сэмплов)<br>
- Чистые данные без пропусков<br>
- Нормализованные числовые фичи<br>
- Сложные нелинейные зависимости (час×погода×сезон)

<h4>🔧 НУЖНО:</h4>
- log1p(cnt) трансформация<br>
- Цикличное кодирование (hr→sin/cos)<br>
- StandardScaler для NN
</h3>

---

## Следующий шаг: Feature Engineering


До log1p:  [1, 40, 142, 281, 977]  ← "хвост" тянет

После:     [0.7, 3.7, 5.0, 5.6, 6.9] ← плавный рост


Модель равномерно учится на всех значениях, а не только на пиках.

In [22]:
# Feature Engineering для NN превосходства
df['hr_sin'] = np.sin(2 * np.pi * df['hr'] / 24)
df['hr_cos'] = np.cos(2 * np.pi * df['hr'] / 24)
df['mnth_sin'] = np.sin(2 * np.pi * df['mnth'] / 12)
df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)

features = ['season', 'yr', 'holiday', 'workingday', 'weathersit', 
           'temp', 'atemp', 'hum', 'windspeed', 
           'hr_sin', 'hr_cos', 'mnth_sin', 'weekday_sin']

X = df[features]
y = np.log1p(df['cnt'])  # Обязательная трансформация!

print("Фичи готовы!")
print(f"X shape: {X.shape}")
print(f"y range: {y.min():.2f} → {y.max():.2f}")
print("\nПервые фичи:")
print(X.head())


Фичи готовы!
X shape: (17379, 13)
y range: 0.69 → 6.89

Первые фичи:
   season  yr  holiday  workingday  weathersit  temp   atemp   hum  windspeed  \
0       1   0        0           0           1  0.24  0.2879  0.81        0.0   
1       1   0        0           0           1  0.22  0.2727  0.80        0.0   
2       1   0        0           0           1  0.22  0.2727  0.80        0.0   
3       1   0        0           0           1  0.24  0.2879  0.75        0.0   
4       1   0        0           0           1  0.24  0.2879  0.75        0.0   

     hr_sin    hr_cos  mnth_sin  weekday_sin  
0  0.000000  1.000000       0.5    -0.781831  
1  0.258819  0.965926       0.5    -0.781831  
2  0.500000  0.866025       0.5    -0.781831  
3  0.707107  0.707107       0.5    -0.781831  
4  0.866025  0.500000       0.5    -0.781831  


<h1 id="feature-engineering" align="center" style="color:#000000;background:#FFFACD;padding:18px;border-radius:15px 50px; font-family:newtimeroman;">
 Feature Engineering: Подготовка данных для превосходства NN
</h1>

<h2 id="new-features" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
Что добавили и зачем
</h2>

<table style="width:100%; border-collapse:collapse;">
<tr style="background:#90EE90;">
    <th>Новые фичи</th><th>Что это</th><th>Зачем для NN</th>
</tr>
<tr>
    <td><b>hr_sin, hr_cos</b></td>
    <td>Час (0-23) → синус/косинус 24ч цикла</td>
    <td><b>КРИТИЧНО:</b> NN поймет, что 23ч ≈ 0ч (ночь), а деревья — нет</td>
</tr>
<tr>
    <td><b>mnth_sin</td>
    <td>Месяц (1-12) → синус 12м цикла</td>
    <td>Зима(1) ≈ зима(12), лето(6-7) пик спроса</td>
</tr>
<tr>
    <td><b>weekday_sin</td>
    <td>День недели (0-6) → синус 7д цикла</td>
    <td>Понедельник ≈ воскресенье (низкий спрос)</td>
</tr>
<tr style="background:#FFF2CC;">
    <td><b>y = log1p(cnt)</b></td>
    <td>Спрос 1-977 → логарифм 0.69-6.89</td>
    <td><b>ОБЯЗАТЕЛЬНО:</b> убирает right skew, стабилизирует градиенты NN</td>
</tr>
</table>

<h2 id="features-ready" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
Готовые данные для обучения
</h2>

<table style="width:100%; border-collapse:collapse;">
<tr style="background:#90EE90;">
    <th>Метрика</th><th>Значение</th><th>Оценка</th>
</tr>
<tr>
    <td><b>X shape</b></td>
    <td><span class="highlight">17,379 × 13 фич</td>
    <td> <b>Отлично:</b> большой объём + средняя размерность</td>
</tr>
<tr style="background:#FFF2CC;">
    <td><b>y range</b></td>
    <td>0.69 → 6.89</td>
    <td> <b>Идеально:</b> log1p сработал, данные готовы для NN</td>
</tr>
<tr>
    <td><b>Цикличные фичи</b></td>
    <td>hr_sin/cos ∈ [-1,1], mnth_sin=0.5 (январь)</td>
    <td> <b>Ключ к победе NN</b> — деревья не поймут цикличность</td>
</tr>
</table>

<h3 style="color:#000000;background:#F0F8FF;padding:12px;border-radius:8px;font-family:monospace;">
ПРОВЕРКА КАЧЕСТВА: hr_sin/cos плавно меняются с часа → NN идеально поймет суточные паттерны
</h3>

<h1 id="split-ready" align="center" style="color:#000000;background:#90EE90;padding:18px;border-radius:15px 50px; font-family:newtimeroman;">
 ДАННЫЕ ГОТОВЫ К ОБУЧЕНИЮ МОДЕЛЕЙ
</h1>

---

## Следующий шаг: Train/Test Split + Scaling


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Train/Test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# StandardScaler (ОБЯЗАТЕЛЬНО для NN!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("=== SPLIT И SCALE ГОТОВЫ ===")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"X_train_scaled mean={X_train_scaled.mean():.3f}, std={X_train_scaled.std():.3f}")
print(f"y_train range: {y_train.min():.2f} → {y_train.max():.2f}")


=== SPLIT И SCALE ГОТОВЫ ===
X_train: (13903, 13)
X_test: (3476, 13)
X_train_scaled mean=0.000, std=1.000
y_train range: 0.69 → 6.88


<h1 id="data-ready" align="center" style="color:#000000;background:#90EE90;padding:18px;border-radius:15px 50px; font-family:newtimeroman;">
 Train/Test Split и Scaling: Идеальная подготовка завершена
</h1>

<h2 id="split-quality" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
Качество разбиения и нормализации
</h2>

<table style="width:100%; border-collapse:collapse;">
<tr style="background:#90EE90;">
    <th>Метрика</th><th>Значение</th><th>Оценка качества</th>
</tr>
<tr>
    <td><b>Train размер</b></td>
    <td><span class="highlight">13,903 сэмпла (80%)</span></td>
    <td> <b>Отлично:</b> достаточно для стабильного обучения NN</td>
</tr>
<tr>
    <td><b>Test размер</b></td>
    <td><span class="highlight">3,476 сэмпла (20%)</span></td>
    <td> <b>Отлично:</b> репрезентативная валидация</td>
</tr>
<tr style="background:#FFF2CC;">
    <td><b>StandardScaler</b></td>
    <td><b>mean=0.000, std=1.000</b></td>
    <td> <b>Идеально:</b> данные готовы для градиентного спуска NN</td>
</tr>
</table>



<h1 id="training-ready" align="center" style="color:#000000;background:#90EE90;padding:18px;border-radius:15px 50px; font-family:newtimeroman;">
 ДАННЫЕ ПОЛНОСТЬЮ ГОТОВЫ К ОБУЧЕНИЮ ТРЁХ МОДЕЛЕЙ
</h1>


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

# LinearRegression — простой baseline
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

# 4 метрики (исправлено для sklearn 1.4+)
lr_r2 = r2_score(y_test, lr_pred)
lr_mae = mean_absolute_error(y_test, lr_pred)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))  # Фикс ошибки!
lr_mape = mean_absolute_percentage_error(y_test, lr_pred)

print("=== LinearRegression РЕЗУЛЬТАТЫ ===")
print(f"R² = {lr_r2:.4f}")
print(f"MAE = {lr_mae:.4f}")
print(f"RMSE = {lr_rmse:.4f}")
print(f"MAPE = {lr_mape:.1%}")


=== LinearRegression РЕЗУЛЬТАТЫ ===
R² = 0.6056
MAE = 0.6979
RMSE = 0.8812
MAPE = 22.4%


<h2 id="lr-results" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
1. LinearRegression — Baseline модель
</h2>

**Результаты на test (3476 сэмплов):**

| Метрика | Значение | Оценка |
|---------|----------|---------|
| **R²** | **60.56%** |  Слабый baseline — линейность покрывает только 60% вариации |
| **MAE** | **0.70** | Средняя ошибка ~20% от диапазона log(cnt) |
| **RMSE** | **0.88** |  Высокий разброс предсказаний |
| **MAPE** | **22.4%** |  Недопустимо для бизнеса |

**Вывод:** Линейная регрессия проваливается на нелинейных зависимостях (час×погода×сезон). Устанавливает низкую планку для RF/NN.

In [26]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

rf_r2 = r2_score(y_test, rf_pred)
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_mape = mean_absolute_percentage_error(y_test, rf_pred)

print("=== RandomForest РЕЗУЛЬТАТЫ ===")
print(f"R² = {rf_r2:.4f}")
print(f"MAE = {rf_mae:.4f}")
print(f"RMSE = {rf_rmse:.4f}")
print(f"MAPE = {rf_mape:.1%}")


=== RandomForest РЕЗУЛЬТАТЫ ===
R² = 0.9472
MAE = 0.2111
RMSE = 0.3224
MAPE = 7.3%


<h2 id="rf-results" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
2. RandomForestRegressor — Классика табличных данных
</h2>

**Результаты на test (3476 сэмплов):**

| Метрика | Значение | Оценка |
|---------|----------|---------|
| **R²** | **94.72%** |  <b>Отличный результат</b> — ансамбль 200 деревьев захватывает нелинейности |
| **MAE** | **0.21** |  <b>Планка качества</b> для сравнения с NN |
| **RMSE** | **0.32** |  <b>Низкий разброс</b> предсказаний |
| **MAPE** | **7.3%** |  <b>Бизнес-приемлемо</b> |

**Вывод:** RandomForest устанавливает высокую планку (R² +34% над LR). Теперь главная цель NN — превзойти 94.72%.

In [27]:
from sklearn.neural_network import MLPRegressor

# Глубокая архитектура для 17k сэмплов
nn = MLPRegressor(
    hidden_layer_sizes=(512, 256, 128, 64),  # 4 слоя
    max_iter=2000, 
    early_stopping=True,
    validation_fraction=0.1,
    random_state=42
)

nn.fit(X_train_scaled, y_train)
nn_pred = nn.predict(X_test_scaled)

nn_r2 = r2_score(y_test, nn_pred)
nn_mae = mean_absolute_error(y_test, nn_pred)
nn_rmse = np.sqrt(mean_squared_error(y_test, nn_pred))
nn_mape = mean_absolute_percentage_error(y_test, nn_pred)

print("=== NeuralNetwork РЕЗУЛЬТАТЫ ===")
print(f"R² = {nn_r2:.4f}")
print(f"MAE = {nn_mae:.4f}")
print(f"RMSE = {nn_rmse:.4f}")
print(f"MAPE = {nn_mape:.1%}")

=== NeuralNetwork РЕЗУЛЬТАТЫ ===
R² = 0.9557
MAE = 0.2088
RMSE = 0.2954
MAPE = 6.8%


<h1 id="final-results" align="center" style="color:#000000;background:#FFFACD;padding:18px;border-radius:15px 50px; font-family:newtimeroman;">
 Итоговое сравнение моделей регрессии Bike Sharing Demand
</h1>

<h2 id="top-models" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
Результаты моделей (test=3476 сэмплов)
</h2>

<table style="width:100%; border-collapse:collapse; font-size:16px;">
<tr style="background:#90EE90;">
    <th><b>🏆 ПОБЕДИТЕЛЬ</b></th>
    <th>R²</th><th>MAE</th><th>RMSE</th><th>Δ vs RF</th>
</tr>
<tr style="background:#E6FFE6;">
    <td><b>NeuralNetwork</b></td>
    <td><b class="highlight">95.57%</b></td>
    <td><b>0.2088</b></td>
    <td><b>0.2954</b></td>
    <td><b>+0.85% R², -8.4% RMSE</b></td>
</tr>
<tr>
    <td><b>RandomForest</b></td>
    <td><b>94.72%</b></td>
    <td>0.2111</td>
    <td>0.3224</td>
    <td>-</td>
</tr>
<tr style="background:#FFE6E6;">
    <td>LinearRegression</td>
    <td>60.56%</td>
    <td>0.6979</td>
    <td>0.8812</td>
    <td>-34.16% R²</td>
</tr>
</table>

<h2 id="key-conclusion" style="color:#000000;background:#FFFACD;padding:12px;border-radius:12px;font-family:newtimeroman;">
    выводы
</h2>
<ul>
<li><b>NeuralNetwork доминирует:</b> +0.85% R² и -8.4% RMSE против RandomForest</li>
<li><b>Ключ к успеху:</b> 17k сэмплов + циклическое кодирование (hr_sin/cos) + архитектура 512→256→128→64</li>
<li><b>RandomForest competitive:</b> но уступает на сложных нелинейных взаимодействиях (время×погода×сезон)</li>
<li><b>LinearRegression провал:</b> -34% R² — линейные модели не справляются</li>
</ul>


<h3 style="color:#000000;background:#FFFACD;padding:8px;border-radius:8px;font-family:newtimeroman;">
**ФИНАЛЬНЫЙ ВЫВОД:** Нейронные сети превосходят RandomForest при ≥10k сэмплов + циклических фичах + нелинейных взаимодействиях
</h3>


1. sin/cos (циклы) — нейронки **ИДЕАЛЬНО** ловят такие паттерны
2. log1p — **стабильные градиенты** (без взрывов/затуханий)
3. StandardScaler — все фичи **mean=0, std=1** → равный вклад

Feature Engineering = перевод данных на язык, который нейронки понимают лучше деревьев.


Из сырых данных (часы 0-23, месяцы 1-12) мы создаём новые признаки, которые компьютер лучше понимает.

Признаки (фичи, или features) — отдельные характеристики или свойства данных, которые используются моделью для обучения и предсказания. То есть это те элементы, которые описывают данные и на основе которых алгоритм делает выводы. Например, в задаче предсказания цены дома площадь, количество комнат и возраст здания будут признаками, которые модель использует для определения стоимости.
источник-https://habr.com/ru/companies/skillfactory/articles/842444/